In [1]:
from googleads import adwords

In [2]:
def traffic_estimate(client):
    traffic_estimator_service = client.GetService(
        'TrafficEstimatorService', version='v201705')
      # Construct selector object and retrieve traffic estimates.
    keywords = [
        {'text': 'mars cruise', 'matchType': 'BROAD'},
        {'text': 'cheap cruise', 'matchType': 'PHRASE'},
        {'text': 'cruise', 'matchType': 'EXACT'}
    ]
    negative_keywords = [
        {'text': 'moon walk', 'matchType': 'BROAD'}
    ]
    keyword_estimate_requests = []
    for keyword in keywords:
        tmp =  {'keyword': {
                'xsi_type': 'Keyword',
                'matchType': keyword['matchType'],
                'text': keyword['text']}}
        keyword_estimate_requests.append(tmp)
        
    for keyword in negative_keywords:
        tmp = { 'keyword': {
                'xsi_type': 'Keyword',
                'matchType': keyword['matchType'],
                'text': keyword['text']
                }, 'isNegative': 'true'}
        keyword_estimate_requests.append(tmp)
    
    # Create ad group estimate requests.
    adgroup_estimate_requests = [{
        'keywordEstimateRequests': keyword_estimate_requests,
        'maxCpc': {
            'xsi_type': 'Money',
            'microAmount': '1000000000'
         }
    }]
    
    # Create campaign estimate requests.
    campaign_estimate_requests = [{
      'adGroupEstimateRequests': adgroup_estimate_requests,
      'criteria': [
          {
              'xsi_type': 'Location',
              'id': '2840'  # United States.
          },
          {
              'xsi_type': 'Language',
              'id': '1000'  # English.
          }
      ],
    }]
    
    # Create the selector.
    selector = {
        'campaignEstimateRequests': campaign_estimate_requests,
    }

    # Optional: Request a list of campaign-level estimates segmented by
    # platform.
    selector['platformEstimateRequested'] = True
    
    # Get traffic estimates.
    estimates = traffic_estimator_service.get(selector)
    campaign_estimate = estimates['campaignEstimates'][0]

    # Display the campaign level estimates segmented by platform.
    if 'platformEstimates' in campaign_estimate:
        platform_template = ('Results for the platform with ID: "%d" and name: '
                         '"%s".')
        for platform_estimate in campaign_estimate['platformEstimates']:
            platform = platform_estimate['platform']
            DisplayEstimate(platform_template % (platform['id'],
                                           platform['platformName']),
                      platform_estimate['minEstimate'],
                      platform_estimate['maxEstimate']) 

    # Display the keyword estimates.
    if 'adGroupEstimates' in campaign_estimate:
        ad_group_estimate = campaign_estimate['adGroupEstimates'][0]
        if 'keywordEstimates' in ad_group_estimate:
            keyword_estimates = ad_group_estimate['keywordEstimates']
            keyword_template = ('Results for the keyword with text "%s" and match '
                          'type "%s":')

            keyword_estimates_and_requests = zip(keyword_estimates,
                                           keyword_estimate_requests)

            for keyword_tuple in keyword_estimates_and_requests:
                if keyword_tuple[1].get('isNegative', False):
                    continue
                keyword = keyword_tuple[1]['keyword']
                keyword_estimate = keyword_tuple[0]
                DisplayEstimate(keyword_template % (keyword['text'],
                                            keyword['matchType']),
                        keyword_estimate['min'], keyword_estimate['max'])
                
def _CalculateMean(min_est, max_est):
    if min_est and max_est:
        return (float(min_est) + float(max_est)) / 2.0
    else:
        return None

def _ConvertDollar(est):
    if est:
        return est/100000.0
    else:
        return None    
    
def _FormatMean(mean):
    if mean:
        return '%.2f' % mean
    else:
        return 'N/A'

def DisplayEstimate(message, min_estimate, max_estimate):
    """Displays mean average cpc, position, clicks, and total cost for estimate.
    Args:
      message: str message to display for the given estimate.
      min_estimate: sudsobject containing a minimum estimate from the
        TrafficEstimatorService response.
      max_estimate: sudsobject containing a maximum estimate from the
        TrafficEstimatorService response.
    """
    # Find the mean of the min and max values.
    mean_avg_cpc = (_CalculateMean(min_estimate['averageCpc']['microAmount'],
                                   max_estimate['averageCpc']['microAmount'])
                    if 'averageCpc' in min_estimate else None)
    mean_avg_pos = (_CalculateMean(min_estimate['averagePosition'],
                                   max_estimate['averagePosition'])
                    if 'averagePosition' in min_estimate else None)
    mean_clicks = _CalculateMean(min_estimate['clicksPerDay'],
                                 max_estimate['clicksPerDay'])
    mean_total_cost = _ConvertDollar(_CalculateMean(min_estimate['totalCost']['microAmount'],
                                     max_estimate['totalCost']['microAmount']))
  
    print message
    print '  Estimated average CPC: %s' % _FormatMean(mean_avg_cpc)
    print '  Estimated ad position: %s' % _FormatMean(mean_avg_pos)
    print '  Estimated daily clicks: %s' % _FormatMean(mean_clicks)
    print '  Estimated daily cost: %s' % _FormatMean(mean_total_cost)

In [3]:
adwords_client = adwords.AdWordsClient.LoadFromStorage()
traffic_estimate(adwords_client)

No handlers could be found for logger "suds.resolver"


Results for the platform with ID: "30000" and name: "Desktop".
  Estimated average CPC: 3638494.50
  Estimated ad position: 1.06
  Estimated daily clicks: 565.90
  Estimated daily cost: 20590.40
Results for the platform with ID: "30001" and name: "Mobile".
  Estimated average CPC: 2604402.00
  Estimated ad position: 1.06
  Estimated daily clicks: 862.79
  Estimated daily cost: 22470.43
Results for the platform with ID: "30002" and name: "Tablet".
  Estimated average CPC: 3554447.00
  Estimated ad position: 1.06
  Estimated daily clicks: 140.52
  Estimated daily cost: 4994.79
Results for the keyword with text "mars cruise" and match type "BROAD":
  Estimated average CPC: 2480399.00
  Estimated ad position: 1.06
  Estimated daily clicks: 0.01
  Estimated daily cost: 0.26
Results for the keyword with text "cheap cruise" and match type "PHRASE":
  Estimated average CPC: 3632216.00
  Estimated ad position: 1.06
  Estimated daily clicks: 345.32
  Estimated daily cost: 12542.89
Results for th

In [4]:
b = adwords.AdWordsClient.LoadFromStorage()
traffic_estimate(b)

Results for the platform with ID: "30000" and name: "Desktop".
  Estimated average CPC: 3638493.50
  Estimated ad position: 1.06
  Estimated daily clicks: 565.90
  Estimated daily cost: 20590.39
Results for the platform with ID: "30001" and name: "Mobile".
  Estimated average CPC: 2604402.00
  Estimated ad position: 1.06
  Estimated daily clicks: 862.79
  Estimated daily cost: 22470.43
Results for the platform with ID: "30002" and name: "Tablet".
  Estimated average CPC: 3554448.00
  Estimated ad position: 1.06
  Estimated daily clicks: 140.52
  Estimated daily cost: 4994.79
Results for the keyword with text "mars cruise" and match type "BROAD":
  Estimated average CPC: 2480399.00
  Estimated ad position: 1.06
  Estimated daily clicks: 0.01
  Estimated daily cost: 0.26
Results for the keyword with text "cheap cruise" and match type "PHRASE":
  Estimated average CPC: 3632214.50
  Estimated ad position: 1.06
  Estimated daily clicks: 345.32
  Estimated daily cost: 12542.89
Results for th